# Loop 21 Analysis: Understanding the Gap

## Current Status
- Best CV: 70.627608 (exp 023_nfp_chebyshev, which fell back to baseline)
- Best LB: 70.6276 (exp_020)
- Target: 68.919154
- Gap: 1.71 points (2.48%)

## Key Observations
1. ALL experiments converge to ~70.63 score
2. C++ SA with 2M iterations only found 3 tiny improvements (0.0003 total)
3. Greedy Chebyshev packing was 2-3x WORSE than baseline
4. The baseline is at an extremely strong local optimum

In [1]:
import pandas as pd
import numpy as np
import json
from shapely.geometry import Polygon
from shapely.affinity import rotate, translate

# Tree template
TREE_TEMPLATE = [
    (0.0, 0.8), (0.125, 0.5), (0.0625, 0.5), (0.2, 0.25), (0.1, 0.25),
    (0.35, 0.0), (0.075, 0.0), (0.075, -0.2), (-0.075, -0.2), (-0.075, 0.0),
    (-0.35, 0.0), (-0.1, 0.25), (-0.2, 0.25), (-0.0625, 0.5), (-0.125, 0.5)
]

def create_tree_polygon(x, y, angle):
    tree = Polygon(TREE_TEMPLATE)
    tree = rotate(tree, angle, origin=(0, 0), use_radians=False)
    tree = translate(tree, x, y)
    return tree

def get_bounding_box_side(trees):
    all_x, all_y = [], []
    for tree in trees:
        minx, miny, maxx, maxy = tree.bounds
        all_x.extend([minx, maxx])
        all_y.extend([miny, maxy])
    return max(max(all_x) - min(all_x), max(all_y) - min(all_y))

print('Functions defined')

Functions defined


In [2]:
# Load current best submission
def parse_s_value(val):
    if isinstance(val, str):
        if val.startswith('s'):
            return float(val[1:])
        return float(val)
    return float(val)

best_df = pd.read_csv('/home/submission/submission.csv')
best_df['x'] = best_df['x'].apply(parse_s_value)
best_df['y'] = best_df['y'].apply(parse_s_value)
best_df['deg'] = best_df['deg'].apply(parse_s_value)
best_df['n'] = best_df['id'].apply(lambda x: int(x.split('_')[0]))

print(f'Loaded {len(best_df)} rows')

# Calculate scores for each N
scores = {}
sides = {}
for n in range(1, 201):
    group = best_df[best_df['n'] == n]
    trees = [create_tree_polygon(row['x'], row['y'], row['deg']) for _, row in group.iterrows()]
    side = get_bounding_box_side(trees)
    sides[n] = side
    scores[n] = (side ** 2) / n

total_score = sum(scores.values())
print(f'Total score: {total_score:.6f}')
print(f'Target: 68.919154')
print(f'Gap: {total_score - 68.919154:.6f} ({(total_score - 68.919154) / 68.919154 * 100:.2f}%)')

Loaded 20100 rows


Total score: 70.627608
Target: 68.919154
Gap: 1.708454 (2.48%)


In [3]:
# Analyze score distribution by N
import matplotlib.pyplot as plt

n_values = list(range(1, 201))
score_values = [scores[n] for n in n_values]
side_values = [sides[n] for n in n_values]

# Calculate theoretical minimum (if trees could be packed perfectly)
# A single tree has area ~0.35 (rough estimate)
# For N trees, minimum area = N * 0.35, so side = sqrt(N * 0.35)
tree_area = 0.35  # approximate
theoretical_min_sides = [np.sqrt(n * tree_area) for n in n_values]
theoretical_min_scores = [(np.sqrt(n * tree_area) ** 2) / n for n in n_values]

print('=== Score Analysis by N Range ===')
for start, end in [(1, 10), (11, 30), (31, 50), (51, 100), (101, 150), (151, 200)]:
    range_score = sum(scores[n] for n in range(start, end+1))
    range_theoretical = sum(theoretical_min_scores[n-1] for n in range(start, end+1))
    print(f'N={start}-{end}: score={range_score:.4f}, theoretical_min={range_theoretical:.4f}, gap={range_score - range_theoretical:.4f}')

=== Score Analysis by N Range ===
N=1-10: score=4.3291, theoretical_min=3.5000, gap=0.8291
N=11-30: score=7.4130, theoretical_min=7.0000, gap=0.4130
N=31-50: score=7.2918, theoretical_min=7.0000, gap=0.2918
N=51-100: score=17.6143, theoretical_min=17.5000, gap=0.1143
N=101-150: score=17.1364, theoretical_min=17.5000, gap=-0.3636
N=151-200: score=16.8430, theoretical_min=17.5000, gap=-0.6570


In [4]:
# Calculate efficiency (how close to theoretical minimum)
efficiencies = []
for n in n_values:
    theoretical = tree_area  # S^2/N = tree_area for perfect packing
    actual = scores[n]
    efficiency = theoretical / actual if actual > 0 else 0
    efficiencies.append(efficiency)

print('\n=== Worst Efficiency N Values (most room for improvement) ===')
worst_n = sorted(zip(efficiencies, n_values))[:20]
for eff, n in worst_n:
    print(f'N={n}: efficiency={eff:.4f}, score={scores[n]:.6f}, side={sides[n]:.4f}')


=== Worst Efficiency N Values (most room for improvement) ===
N=1: efficiency=0.5293, score=0.661250, side=0.8132
N=2: efficiency=0.7764, score=0.450779, side=0.9495
N=3: efficiency=0.8051, score=0.434745, side=1.1420
N=5: efficiency=0.8396, score=0.416850, side=1.4437
N=4: efficiency=0.8402, score=0.416545, side=1.2908
N=7: efficiency=0.8752, score=0.399897, side=1.6731
N=6: efficiency=0.8759, score=0.399610, side=1.5484
N=9: efficiency=0.9034, score=0.387415, side=1.8673
N=8: efficiency=0.9081, score=0.385407, side=1.7559
N=15: efficiency=0.9284, score=0.376978, side=2.3780
N=10: efficiency=0.9293, score=0.376630, side=1.9407
N=21: efficiency=0.9297, score=0.376451, side=2.8117
N=20: efficiency=0.9307, score=0.376057, side=2.7425
N=22: efficiency=0.9327, score=0.375258, side=2.8733
N=11: efficiency=0.9335, score=0.374924, side=2.0308
N=16: efficiency=0.9355, score=0.374128, side=2.4466
N=26: efficiency=0.9358, score=0.373997, side=3.1183
N=12: efficiency=0.9390, score=0.372724, side

In [5]:
# Calculate how much improvement we need per N to reach target
target = 68.919154
current = total_score
gap = current - target

print(f'\n=== Gap Analysis ===')
print(f'Current: {current:.6f}')
print(f'Target: {target:.6f}')
print(f'Gap: {gap:.6f}')
print(f'Average improvement needed per N: {gap / 200:.6f}')

# If we improved the worst N values by X%, what would the total be?
print('\n=== What-If Analysis ===')
for improvement_pct in [5, 10, 15, 20, 25, 30]:
    # Improve worst 50 N values by this percentage
    worst_50 = sorted([(scores[n], n) for n in n_values], reverse=True)[:50]
    improved_score = current
    for score, n in worst_50:
        improved_score -= score * (improvement_pct / 100)
    print(f'Improve worst 50 N by {improvement_pct}%: {improved_score:.4f} (gap: {improved_score - target:.4f})')


=== Gap Analysis ===
Current: 70.627608
Target: 68.919154
Gap: 1.708454
Average improvement needed per N: 0.008542

=== What-If Analysis ===
Improve worst 50 N by 5%: 69.6748 (gap: 0.7556)
Improve worst 50 N by 10%: 68.7220 (gap: -0.1972)
Improve worst 50 N by 15%: 67.7691 (gap: -1.1500)
Improve worst 50 N by 20%: 66.8163 (gap: -2.1028)
Improve worst 50 N by 25%: 65.8635 (gap: -3.0557)
Improve worst 50 N by 30%: 64.9107 (gap: -4.0085)


In [6]:
# Look at the actual improvements found by C++ SA
print('\n=== Recent Improvements from C++ SA ===')
with open('/home/code/experiments/022_cpp_sa_5M_targeted/metrics.json') as f:
    metrics = json.load(f)

print(f"Total improvement: {metrics['total_improvement']:.6f}")
print(f"Improvements found: {metrics['improvements']}")
for n, imp in metrics['improvement_details']:
    print(f"  N={n}: {imp:.6f} ({imp/scores[n]*100:.4f}% of N={n} score)")

print('\n=== Conclusion ===')
print('The C++ SA with 2M iterations found only 0.0003 improvement.')
print('At this rate, reaching the target would require ~5,700 more runs.')
print('We need a FUNDAMENTALLY DIFFERENT approach, not more SA iterations.')


=== Recent Improvements from C++ SA ===
Total improvement: 0.000262
Improvements found: 3
  N=35: 0.000217 (0.0593% of N=35 score)
  N=64: 0.000011 (0.0031% of N=64 score)
  N=88: 0.000034 (0.0098% of N=88 score)

=== Conclusion ===
The C++ SA with 2M iterations found only 0.0003 improvement.
At this rate, reaching the target would require ~5,700 more runs.
We need a FUNDAMENTALLY DIFFERENT approach, not more SA iterations.
